### 목표: RAG Q&A 동작 구축 → 그 다음에 LangGraph 연결 → 마지막에 Memory/ToolNode 확장

#### 2단계: LangGraph 연결
- 1단계에서 만든 RAG QnA를 Class로 정리
- LangGraph Node로 연결 테스트

---

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import StateGraph, START, END
from langchain.tools import tool
from functools import partial
from pinecone import Pinecone, ServerlessSpec
from typing import TypedDict, Annotated, Optional, Literal, List
from dotenv import load_dotenv 
import operator
import os


load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "plant-qna"
index = pc.Index(index_name)


@tool
def tool_rag_qna(query: str) -> str:
    """식물 상담 QnA 전용 RAG 도구"""
    vector_store = PineconeVectorStore(
        index=index, 
        embedding=OpenAIEmbeddings(model="text-embedding-3-small")
    )

    retriever = vector_store.as_retriever(search_kwargs={"k": 3})   
    retrievals = retriever.batch([query]) 
    
    return str(retrievals[0])


class ModelQna:
    def __init__(self, tools):        
        self.tools = tools      

    def get_response(self, messages):
        
        prompt = """
        너는 식물에 대해 차분하게 상담해 주는 전문가이다.
        아래 형식을 반드시 지키되, 실제 상담사가 말하듯 자연스럽고 단정적인 말투로 작성한다.

        ### 답변 방식 ###
        - 첫 문장은 사용자의 고민에 대한 핵심 답변을 한 줄로 요약한다. (채팅 응답처럼)
        - 이후 이어지는 RAG 정보는 비슷한 사례의 해결 방향을 '요약 3줄'로 정리한다.
        - 모든 문장은 따뜻하지만 과하지 않게, 실제 상담사가 말하듯 단정적으로 말한다.
        - 마지막 문장은 대화를 이어가기 위해 질문형으로 마무리한다.
        
        ### 출력 형식 ###
        [사용자의 상황을 판단해서 가장 핵심적인 조언을 한 문장으로 제시]
        [현재 상황에 맞는 다음 추가 질문 유도]
        """
        
        system_msg = SystemMessage(prompt)
        input_msg = [system_msg] + messages

        model = ChatOpenAI(
            model="gpt-4o-mini", 
            temperature=0.3
        ).bind_tools(self.tools)

        response = model.invoke(input_msg)
        
        return response

- 랭그래프 연결은 준영님